In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from random import choice
from IPython.display import clear_output

#### Projeto 2 - Organizando uma eleição

In [8]:
#cadastrar todos os apartamentos primeiro
class Apartamento:
    '''
    Cria o dicionário de dados usando como chave o 
    número do apartamento e os valores são os nomes
    dos moradores (lista).
    '''
    def __init__(self,numero_apartamento):
        '''
        Construtor do apartamento, a identificação será uma 
        string pois o condomínio pode conter vários blocos e
        a numerção poderá repetir. ex: 101A , 101B.
        
        Parametros:
        numero_apartamento : str
            Chave do dicionário.
        
        '''
        self._lista_moradores = []
        self._status_votacao = False
        self._numero_apartamento = numero_apartamento
    
    def __repr__(self):
        return f'Nº Apartamento: {self._numero_apartamento},Moradores: {self._lista_moradores}, Votou?: {self._status_votacao}'

    def adiciona_morador(self,morador):
        '''
        Adciciona objeto Morador a lista de moradores do partamento.
        Caso o input não seja objeto Morador imprime mensagem de erro.
        
        Parâmetros:
        
        morador : Morador          
        
            Objeto da classe Morador.
        '''
        if(isinstance(morador,Morador)):
            self._lista_moradores.append(morador)
            morador._apartamento = self
        else:
            print("Morador inválido. Não é um objeto Morador")
    
    def lista_moradores(self):
        '''
        Imprime lista de moradores do apartamento.
        '''
        lista_moradores = [{"Apartamento "+str(self._numero_apartamento): morador._nome} for morador in self._lista_moradores]
        df = pd.DataFrame(lista_moradores)
        display(df) 

class Morador:
    '''
    Define os atributos dos indivíduos do condomínio. 
    
    '''
    
    def __init__(self, nome):
        '''
        Costrutor do objeto Morador com nome e variavel apartamento que irá ser usada na classe Apartamento.
        
        Parâmetros;
        
        nome : str
            Nome do indivíduo que será usado como identificação do objeto Morador
        
        
        '''
        if(isinstance(nome,str)):
            self._nome = nome
            self._apartamento = None
        else:
            print("O nome deve ser uma string")

      
    def __repr__(self):
        '''
        Método mágico para melhor visualização objeto Morador.
        '''
        return f'Morador: {self._nome}, Apartamento: {self._apartamento}'
    
    def vota_candidato(self,urna,numero_candidato):
        if(isinstance(urna,Urna)):
            if(isinstance(numero_candidato,int)):
                if self._apartamento._status_votacao:
                    print("Voto do apartamento já contabilizado. Não é possível voltar 2 vezes.")
                    return False
                else:
                    return urna.executa_voto(self._apartamento,numero_candidato)
            else:
                print("Nº do candidato inválido.")
        else:
            print("Urna inválida. Não é um objeto Urna")
                
class Candidato(Morador):
    '''
    Classe filha da classe Morador.
    Estabelece o morador escolhido como candidato a síndico.
    
    '''
    def __init__(self,morador):
        '''
        Construtor de candidato usando um nome válido de morador. 
        Caso o nome não esteja na lista de moradores imprime mensagem de erro.  
        
        Parâmetros:
        
        morador : str
        
            Nome do morador que se candidatou ao cargo de síndico.
        '''
        
        if(isinstance(morador,Morador)):
            self._nome = morador._nome
            self._apartamento = morador._apartamento
            self._numero_urna = 0 #escolha aleatória entre 0 - 100
            self._contagem_votos = 0
        else:
            print("Morador inválido. Não é um objeto Morador")

    def __repr__(self):
        '''
        Método mágico para melhor visualização objeto Morador.
        '''
        return f'Candidato: {self._nome}, Número Urna: {self._numero_urna}'

    def atualiza_votos(self):
        '''
        Atualização no contador de voctos do candidato.
        Requisitado na função executa_voto.
        '''
        self._contagem_votos = self._contagem_votos + 1
    
   
    
class Urna:
    '''
    Define o funcionamento da votação.
    '''
    def __init__(self):
        '''
        Construtor da classe urna que recebe informação das classes
        Apartamento, Morador, Candidato e inicia a contagem dos votos.
        
        '''
        self._lista_candidatos = dict()
        self._lista_apartamentos = []
        
    def cadastra_apartamento(self,apartamento):
        if(isinstance(apartamento,Apartamento)):
            self._lista_apartamentos.append(apartamento)
        else:
            print("Apartamento inválido. Não é um objeto Apartamento")
            
    def cadastra_candidato(self,candidato):
        if(isinstance(candidato,Candidato)):
            numero_candidato = choice([i for i in range(1,100) if i not in self._lista_candidatos.keys()])#
            candidato._numero_urna = numero_candidato
            self._lista_candidatos.update({numero_candidato:candidato})
        else:
            print("Candidato inválido. Não é um objeto Candidato")
            
    def executa_voto(self,apartamento,numero_candidato):
            if not isinstance(numero_candidato,int):
                print("Dado inválido. Deve ser fornecido um número")
            elif not isinstance(apartamento,Apartamento):
                print("Apartamento inválido. Não é um objeto Apartamento")
            elif numero_candidato not in self._lista_candidatos.keys():
                print("Nº de candidato não registrado.")
            elif apartamento not in self._lista_apartamentos:
                print("Apartamento não habilitado para votar.")
            else:
                self._lista_candidatos[numero_candidato].atualiza_votos()###
                apartamento._status_votacao = True
                self._lista_apartamentos.remove(apartamento)
                print(self._lista_apartamentos)
                if(len(self._lista_apartamentos) == 0):
                    return True
            return False
        
    
    def imprime_resultado(self):
        '''
        Plota um gráfico de barras com a contagem dos votos.
        '''
        candidatos = [candidato._nome for candidato in self._lista_candidatos.values()]
        votos = [candidato._contagem_votos for candidato in self._lista_candidatos.values()]
        plt.bar(candidatos, votos)
        plt.title('Resultado das eleições')
        plt.xlabel('Candidatos')
        plt.ylabel('Votos')
        
    def mostra_candidatos(self):
        '''
        Imprime o painel com a numeração que deverá ser usada na urna.
        '''
        lista_candidato = [{"Candidato": candidato._nome,"Nº Urna": key} for key,candidato in self._lista_candidatos.items()]
        df = pd.DataFrame(lista_candidato)
        display(df)

In [30]:
moradores_do_predio = pd.read_csv('moradores2.csv')  
#moradores_do_predio = pd.DataFrame(moradores_do_predio)
display(moradores_do_predio)


,Nome do Morador;\tApartmento;\tCandidato
0,Joyce;\t10a;\tTrue
1,Matheus;\t101;b\tFalse
2,Rafael;\t10c;\tFalse
3,Luis;\t01a;\tTrue
4,Camila;\t20b;\tFalse
5,Andr‚\t;201c\t;False
6,JoÆo\t;202c\t;False


In [ ]:
#lê arquivo csv de apartamentos
apartamentos = pd.read_csv('apartamentos.csv', encoding = 'utf8')

#cria a variavel que vai guardar os apartamentos
apartamentos_existentes = {}

for apartamento in apartamentos.values:
    nome_app = str(apartamento[0]).upper()
    apartamentos_existentes[nome_app] = Apartamento(nome_app)

In [ ]:
#cria moradores e candidatos
moradores_existentes = []
candidatos_existentes = []

for morador in moradores_do_predio.values:
    novo_morador = Morador(morador[0])
    moradores_existentes.append(novo_morador)
    apmorador = str(morador[1]).upper()
    apartamentos_existentes[apmorador].adiciona_morador(novo_morador)
    #verifica se o morador é candidato
    if morador[2]:
        candidatos_existentes.append(Candidato(novo_morador))

In [ ]:
for apartamento in apartamentos.values:
    nome_app = str(apartamento[0]).upper()
    apartamentos_existentes[nome_app] = Apartamento(nome_app)

In [ ]:
#lista moradores de cada apartamento
for apartamento in apartamentos_existentes.values():
    apartamento.lista_moradores()

In [ ]:
#lista os candidatos
lista_candidato = [{"Candidato": candidato._nome,"Nº Urna": candidato._numero_urna} for candidato in candidatos_existentes]
df = pd.DataFrame(lista_candidato)
display(df)

In [ ]:
#criar urna
urna  = Urna()

In [ ]:
#cadastrar apartamentos na urna
for apartamento in apartamentos_existentes.values():
    #verifica se apartamento não está vazio
    if len(apartamento._lista_moradores) > 0:
        urna.cadastra_apartamento(apartamento)

In [ ]:
#cadastrar candidatos
for candidato in candidatos_existentes:
    urna.cadastra_candidato(candidato)

In [ ]:
urna.mostra_candidatos()

In [ ]:
terminou = False
display(moradores_do_predio)
while terminou == False:
    quemVaiVotar = int(input('Digite o id de qual morador vai votar: '))
    print()
    print('Olá ',moradores_existentes[quemVaiVotar]._nome)
    print('\nSelecione um candidato abaixo:')
    urna.mostra_candidatos()
    seuCandidato = int(input('Digite o número do seu candidato: '))
    clear_output(wait=True)
    terminou = moradores_existentes[quemVaiVotar].vota_candidato(urna,seuCandidato)
    display(moradores_do_predio)
urna.imprime_resultado()

In [31]:
urna.imprime_resultado()



def graficoVotos(candidatos, votos):
    plt.bar(candidatos, votos)
    plt.title('Resultado das eleições')
    plt.xlabel('Candidatos')
    plt.ylabel('Votos')

NameError: name 'urna' is not defined